In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd 
import seaborn as sns 
import statistics
import numpy as np
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

In [4]:
%matplotlib inline

In [5]:
modeldf2019 = pd.read_csv('mydata/MMSA2019_train.csv')
modeldf2021 = pd.read_csv('mydata/MMSA2021_train.csv')

In [6]:
#pd.set_option('display.max_columns', None)

In [7]:
pd.set_option('display.max_info_columns', 1000)
pd.set_option('display.max_info_rows', 1000000)

In [8]:
import io
buffer = io.StringIO()
modeldf2019.info(buf=buffer)
info_str = buffer.getvalue()
print(info_str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140514 entries, 0 to 140513
Data columns (total 180 columns):
 #    Column      Non-Null Count   Dtype  
---   ------      --------------   -----  
 0    Unnamed: 0  140514 non-null  int64  
 1    DISPCODE    140514 non-null  int64  
 2    STATERE1    47593 non-null   float64
 3    CELPHONE    47593 non-null   float64
 4    LADULT1     47593 non-null   float64
 5    COLGSEX     11 non-null      float64
 6    LANDSEX     20189 non-null   float64
 7    RESPSLCT    24008 non-null   float64
 8    SAFETIME    92921 non-null   float64
 9    CADULT1     92921 non-null   float64
 10   CELLSEX     92917 non-null   float64
 11   HHADULT     92918 non-null   float64
 12   SEXVAR      140514 non-null  int64  
 13   GENHLTH     140501 non-null  float64
 14   PHYSHLTH    140503 non-null  float64
 15   MENTHLTH    140508 non-null  float64
 16   POORHLTH    77558 non-null   float64
 17   HLTHPLN1    140510 non-null  float64
 18   PERSDOC2    140511 non

In [25]:
buffer = io.StringIO()
modeldf2021.info(buf=buffer)
info_str = buffer.getvalue()
print(info_str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151861 entries, 0 to 151860
Data columns (total 153 columns):
 #    Column    Non-Null Count   Dtype   
---   ------    --------------   -----   
 0    DISPCODE  151861 non-null  category
 1    STATERE1  37094 non-null   category
 2    CELPHON1  37094 non-null   category
 3    LADULT1   37094 non-null   category
 4    COLGSEX   13 non-null      category
 5    LANDSEX   15568 non-null   category
 6    RESPSLCT  20603 non-null   category
 7    SAFETIME  114767 non-null  category
 8    CADULT1   114767 non-null  category
 9    CELLSEX   114766 non-null  category
 10   HHADULT   114766 non-null  float64 
 11   SEXVAR    151861 non-null  category
 12   GENHLTH   151860 non-null  category
 13   PHYSHLTH  151859 non-null  float64 
 14   MENTHLTH  151861 non-null  float64 
 15   POORHLTH  81991 non-null   float64 
 16   PRIMINSR  151860 non-null  category
 17   PERSDOC3  151861 non-null  category
 18   MEDCOST1  151860 non-null  category
 19   

In [26]:
modeldf2021.columns

Index(['DISPCODE', 'STATERE1', 'CELPHON1', 'LADULT1', 'COLGSEX', 'LANDSEX',
       'RESPSLCT', 'SAFETIME', 'CADULT1', 'CELLSEX',
       ...
       '_VEGLT1A', '_FRT16A', '_VEG23A', '_FRUITE1', '_VEGETE1', '_MMSA',
       '_MMSAWT', 'SEQNO', 'MMSANAME', 'STATE'],
      dtype='object', length=153)

### Data Cleaning and Merging 

In [11]:
#Any variable with value counts below 14 with be turned to a categorical nominal datatype:
# Convert 'col1' from float to categorical
for col in modeldf2019.columns:
    if len(modeldf2019[col].value_counts()) < 15:
        modeldf2019[col] = modeldf2019[col].astype('category')
        
for col in modeldf2021.columns:
    if len(modeldf2021[col].value_counts()) < 15:
        modeldf2021[col] = modeldf2021[col].astype('category')
               


In [12]:
#modeldf2019.info()

In [13]:
#modeldf2021.info()

In [14]:
##Extract state from MMSANAME

def get_state(col):
    return col.split(',')[1]

modeldf2019['STATE'] = modeldf2019['MMSANAME'].apply(get_state)

modeldf2021['STATE'] = modeldf2021['MMSANAME'].apply(get_state)

In [92]:
columns = ['FRNCHDA_','POTADA1_', 'FRUTDA2_', 'FTJUDA2_', 'VEGEDA2_', 'GRENDA1_', 
                '_FRUTSU1', '_VEGESU1', '_HLTHPLN','PRIMINSR', '_RACE', 'MEDCOST1', 'MARITAL', '_EDUCAG', 
                'RENTHOM1', 'EMPLOY1', 'CHILDREN', '_INCOMG1', '_TOTINDA', 'CHCOCNCR', 'SMOKE100', 
                'SMOKDAY2', 'USENOW3','_SMOKER3', '_RFSMOK3','_RFBING5', 'DIABETE4', 
                'CHCOCNCR', '_MICHD', '_RFHYPE6', '_RFCHOL3', 'ADDEPEV3', 'DECIDE', '_TOTINDA', '_AGEG5YR', 
                'WTKG3', '_BMI5', '_BMI5CAT', '_SEX','STATE','SEQNO']
len(columns)

41

In [93]:
#RENAME COLUMNS:
modeldf2019.rename(columns={'_INCOMG':'_INCOMG1','_RFHYPE5':'_RFHYPE6','HLTHPLN1': 'PRIMINSR','MEDCOST':'MEDCOST1',
                  '_RFCHOL2':'_RFCHOL3'},inplace=True)


In [94]:
#Create a _HLTHPLN from PRIMINSR IN 2019 df (- '_HLTHPLN' - Categorical variable for healthcare plan )

modeldf2019['_HLTHPLN'] = modeldf2019['PRIMINSR'].apply(lambda x: 1 if x in [1,2,3,4,5,6,7,8,9] else 2 if x == 88 else 'NA')

modeldf2019['_TOTINDA'] = modeldf2019['_TOTINDA'].astype(float)

#Create a DROCDY3_ from ALCDAY5 by dividing the ALCDAY5 variable by 7 days per week or 30 days per month

def compute_drocdy3_(x):
    # Handle NaN values
    if pd.isna(x):
        return np.nan
    
    x_int = int(str(x).split(".")[0])
    
    if x_int == 888:
        return 0.0
    elif x_int // 100 == 1:
        return (x_int % 100) / 7.0
    elif x_int // 100 == 2:
        return (x_int % 100) / 30.0
    elif x_int in [777, 999]:
        return np.nan
    else:
        return float(x_int)

modeldf2021['DROCDY3_'] = modeldf2019['ALCDAY5'].apply(compute_drocdy3_)

In [98]:
modeldf2019_2 = modeldf2019[columns]
modeldf2021_2 = modeldf2021[columns]

In [99]:
model_df = pd.concat([modeldf2019_2,modeldf2021_2])

In [100]:
model_df.head()

,FRNCHDA_,POTADA1_,FRUTDA2_,FTJUDA2_,VEGEDA2_,GRENDA1_,_FRUTSU1,_VEGESU1,_HLTHPLN,PRIMINSR,...,ADDEPEV3,DECIDE,_TOTINDA,_AGEG5YR,WTKG3,_BMI5,_BMI5CAT,_SEX,STATE,SEQNO
0,0.0,0.0,100.0,33.0,50.0,50.0,133.0,100.0,1.0,1.0,...,1.0,2.0,1.0,7.0,11294.0,3786.0,4.0,1.0,WI,2.019001e+09
1,0.0,10.0,200.0,0.0,14.0,29.0,200.0,53.0,1.0,1.0,...,2.0,2.0,1.0,13.0,11113.0,3232.0,4.0,1.0,IA-NE-SD,2.019000e+09
2,0.0,14.0,200.0,0.0,100.0,100.0,200.0,214.0,1.0,1.0,...,2.0,2.0,2.0,11.0,13154.0,3723.0,4.0,1.0,FL,2.019000e+09
3,10.0,10.0,0.0,0.0,100.0,13.0,0.0,133.0,1.0,1.0,...,2.0,2.0,2.0,9.0,6895.0,2969.0,3.0,2.0,RI-MA,2.019003e+09
4,43.0,29.0,14.0,0.0,29.0,0.0,14.0,101.0,1.0,1.0,...,2.0,2.0,1.0,6.0,9072.0,3132.0,4.0,2.0,KS,2.019002e+09


In [102]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292375 entries, 0 to 151860
Data columns (total 41 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   FRNCHDA_  262098 non-null  float64 
 1   POTADA1_  259732 non-null  float64 
 2   FRUTDA2_  262705 non-null  float64 
 3   FTJUDA2_  262390 non-null  float64 
 4   VEGEDA2_  259252 non-null  float64 
 5   GRENDA1_  262418 non-null  float64 
 6   _FRUTSU1  258320 non-null  float64 
 7   _VEGESU1  252218 non-null  float64 
 8   _HLTHPLN  292371 non-null  float64 
 9   PRIMINSR  292370 non-null  float64 
 10  _RACE     292373 non-null  category
 11  MEDCOST1  292371 non-null  category
 12  MARITAL   292353 non-null  category
 13  _EDUCAG   292375 non-null  float64 
 14  RENTHOM1  292361 non-null  category
 15  EMPLOY1   289981 non-null  category
 16  CHILDREN  288326 non-null  float64 
 17  _INCOMG1  292375 non-null  float64 
 18  _TOTINDA  292375 non-null  float64 
 19  CHCOCNCR  292371 non-nu

### Generate EDA Report with Original Dataset

In [ ]:
profile = ProfileReport(modeldf2019)
profile.to_file(output_file="mydata/EDA_Report_2019.html")

profile = ProfileReport(modeldf2021)
profile.to_file(output_file="mydata/EDA_Report_2021.html")

### Handling Missing Data

In [ ]:
#Step1 Remove Columns that have more than 50% missing values

def columns_with_high_null_percentage(df, threshold=0.9):
    """
    Get the columns with null values exceeding the specified threshold in a DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        threshold (float): The threshold for null values (default is 0.9, meaning 90%).

    Returns:
        List[str]: A list of column names with null values exceeding the threshold.
    """
    null_percentages = (df.isnull().sum() / len(df)).sort_values(ascending=False)
    high_null_columns = null_percentages[null_percentages > threshold].index.tolist()
    return high_null_columns

high_null_columns = columns_with_high_null_percentage(model_build_df, threshold=0.4)
print(high_null_columns)


In [ ]:
model_build_df_1 = model_build_df.drop(columns=high_null_columns)
model_build_df_1.info()

In [ ]:
sns.countplot(x=model_build_df_1['_RFHLTH'])

In [ ]:
#number with any chronic illness saying they are in good health 

#cancer CHCOCNCR 1
#diabetes  DIABETE4 1
#heart disease _MICHD 1
#high bp _RFHYPE6 2

df_goodhealth = model_build_df_1[model_build_df_1['_RFHLTH'] == 1]

x = len(df_goodhealth)

print ("number of those that say they are in good health", x)

df_chronic_GH = df_goodhealth[(df_goodhealth['CHCOCNCR'] == 1) | 
              (df_goodhealth['DIABETE4'] == 1) | 
              (df_goodhealth['_MICHD'] == 1) |
              (df_goodhealth['_RFHYPE6']==2)]

print('% of those that say they are in good health and have at least 1 chronic disease',len(df_chronic_GH)/x)

In [ ]:
#Select columns of interest for each research question and for prediction of ones percieved GENHLTH

In [ ]:
#Step4 Remove Highly correlated Columns 
plt.figure(figsize=(20,15))
num_cols = model_build_df_1.select_dtypes(exclude='category').columns
sns.heatmap(model_build_df_1[num_cols].corr(),cmap='viridis',annot=True)

In [ ]:
#COLUMNS WITH HIGH CORRELATION

_FRUTSU1(TOTAL FRUITS CONSUMED PER DAY) AND FRUTDA2

In [ ]:
#profile = ProfileReport(model_build_df)
#profile.to_file(output_file="mydata/EDA_Report_No_NA.html")

## ANSWERING THE RESEARCH QUESTIONS 

How do dietary habits and nutrition shape an individual's health outcome?
- we know that studies have shown that high fiber intake is associated with lower mortality.
as seen in this comparative study https://pubmed.ncbi.nlm.nih.gov/22648726/. 
- we would like to see how a high vegetable/fruit diet may be associated with a person percieved health. 
- the darker the vegetables the higher the fibre content

In [ ]:
#Columns of interest would be nutrition and diet related columns:
nutrition_cols = []
for key, value in description_dict.items():
    if any(sub in str(value).lower() for sub in ['fruit','vegetable','fruits','vegetables']):
        nutrition_cols.append(key)

In [ ]:
nutrition_cols

In [ ]:
#drop alcolhol related columns 
nutrition_cols = nutrition_cols[2:]
for col in nutrition_cols:
    print(description_dict[col])

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
#make a dataframe by combining nutrition columns with health status, health days columns  
nutrition_cols = nutrition_cols + ['GENHLTH','PHYSHLTH','POORHLTH','_RFHLTH','_PHYS14D','_MENT14D']
df_nutrition = model_build_df[nutrition_cols]

In [ ]:
df_nutrition.head()

In [ ]:
plt.figure(figsize=(15,10))
ax = sns.heatmap(df_nutrition.corr(),cmap='viridis',annot=True, annot_kws={"size": 8})

### Looking at the data we could use the calculated Total calcualted variables per day for the fruits and vegetables to see its relation to general health 

In [ ]:
#_FRUTSU1=(FTJUDA2_/100) + (FRUTDA2_/100); 
#_FRUTSU1=round((_FRUTSU1*100),1); 

#WE CAN MAKE A TOTAL FRUITS PER DAY COLUMN 
df_nutrition['Total_fruits_daily'] = df_nutrition['_FRUTSU1'].apply(lambda x: round(x/100,1) )
df_nutrition.head()

In [ ]:
df_nutrition['Total_fruits_daily'].value_counts().head(20)

In [ ]:
#_VEGESU1=(GRENDA1_/100) + (FRNCHDA_/100) + (POTADA1_/100) + 
#(VEGEDA2_/100); 
#_VEGESU1=round((_VEGESU1*100),1); 

#WE CAN MAKE A TOTAL VEGETABLES PER DAY COLUMN 
df_nutrition['Total_vegetables_daily'] = df_nutrition['_VEGESU1'].apply(lambda x: round(x/100,1))
df_nutrition.head()

In [ ]:
df_nutrition['Total_vegetables_daily'].value_counts().head(20)

In [ ]:
df_nutrition.describe([0.25,0.5,0.75,0.99]).T

In [ ]:
#lets filter data to remove outliers 
filtered_df = df_nutrition[(df_nutrition['Total_fruits_daily'] <= 9.0) & 
                           (df_nutrition['Total_vegetables_daily'] <= 26.6396)]

In [ ]:
len(filtered_df)

In [ ]:
#Distribution of Total_fruits_daily
sns.histplot(x=filtered_df['Total_vegetables_daily'])

In [ ]:
#Distribution of Total_vegetables_daily
sns.histplot(x=filtered_df['Total_fruits_daily'])

In [ ]:
#create a BOX PLOT FOR _RFHLTH Vs Total fruits 
plt.figure(figsize=(5,5))
sns.set_style('darkgrid')
sns.boxplot(data=filtered_df, x='Total_fruits_daily', y='_RFHLTH')

In [ ]:
#CREATE A SCATTERPLOT FOR PHYSHLTH(Number of days of poor physical health) Vs Total fruits consued daily
plt.figure(figsize=(5,5))
sns.set_style('darkgrid')
sns.boxplot(data=filtered_df, y='Total_fruits_daily', x='_PHYS14D')

In [ ]:
#create a BOX PLOT FOR _RFHLTH Vs Total_vegetables_daily
plt.figure(figsize=(5,5))
sns.set_style('darkgrid')
sns.boxplot(data=filtered_df, x='Total_vegetables_daily', y='_RFHLTH')

In [ ]:
#Lets create a pivot table showing the mean fruits and vegetables consumed per day for each _RFHLTH group 

pivot_result = pd.pivot_table(filtered_df, values=['Total_fruits_daily','Total_vegetables_daily'],index='_RFHLTH', aggfunc='mean')

#Only looking at those who reported Good health vs those that reported poor/bad health
pivot_result[0:2]

- This analysis shows that those who reported good or better health had a higher mean daily intake of fruits and vegetables. Not a significant difference.

### Apply association rules between eating fruits and vegetables and good percieved health

In [ ]:
#!pip install mlxtend

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Assuming you already have a DataFrame called filtered_df with 'Total_fruits_daily', 'Total_vegetables_daily' and '_RFHLTH' columns

# Binning Total_fruits_daily
fruit_bins = [0, 2, 4, 10]  # Adjust bins as per your data distribution for fruits
fruit_labels = ['Low_Fruit', 'Medium_Fruit', 'High_Fruit']
filtered_df['Fruit_Intake_Binned'] = pd.cut(filtered_df['Total_fruits_daily'], bins=fruit_bins, labels=fruit_labels, include_lowest=True)

# Binning Total_vegetables_daily
veg_bins = [0, 8, 16, 26]  # Adjust bins as per your data distribution for vegetables
veg_labels = ['Low_Veg', 'Medium_Veg', 'High_Veg']
filtered_df['Veg_Intake_Binned'] = pd.cut(filtered_df['Total_vegetables_daily'], bins=veg_bins, labels=veg_labels, include_lowest=True)

# Convert to the binary format
filtered_df_binary = pd.get_dummies(filtered_df[['Fruit_Intake_Binned', 'Veg_Intake_Binned', '_RFHLTH']])

# Apply Apriori to find frequent itemsets
frequent_itemsets = apriori(filtered_df_binary, min_support=0.1, use_colnames=True)

# Generate Association Rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

What is the role of access to healthcare,marriage status, Education and income in predicting health outcomes?

In [ ]:
#Get the columns related to access to healthcare,

In [ ]:
#make a dataframe by combining acess to healthcare columns with health status, health days columns  

In [ ]:
#Do some EDA analysis and correlation analysis to see if there are relationships btw acess to healthcare and percieved
#health

In [ ]:
#Apply association rules between acess to healthcare and good percieved health

Can lifestyle factors, including smoking and alcohol consumption, 
predict health risks and outcomes?

In [ ]:
#Get the columns related to alcohol consumption and smoking

In [ ]:
#Get the columns related to exercise

How predictable are chronic diseases, such as diabetes and hypertension, through a combined analysis of 
lifestyle factors and genetic predisposition?

What is the significant contribution of mental health factors, such as stress, anxiety, 
and depression, to health outcomes, and what pathways influence overall well-being?